# Problem klasyfikacji gęstości (DCP)

### Importy

In [1]:
import math
import numpy as np
from operator import itemgetter

Funkcje generujące diagram czasoprzestrzenny:

In [2]:
def eca_get_lut(rule_num: int) -> np.ndarray:
    return np.array([int(x) for x in bin(rule_num)[2:].zfill(8)], dtype=np.uint8)

def eca_evolve(lut: np.ndarray, x: np.ndarray) -> np.ndarray:
    return lut[7 - (np.roll(x, 1) * 4 + x * 2 + np.roll(x, -1))]

def eca_evolve_spacetime(lut: np.ndarray, initial_conf: np.ndarray, steps: int) -> np.ndarray:
    rows = [initial_conf]
    for _ in range(1, steps):
        rows.append(eca_evolve(lut, rows[-1]))
    return np.stack(rows)


Funkcja do losowania konfiguracji początkowej:

In [3]:
def generate_random_initial_configuration(length: int) -> np.ndarray:
    return np.random.randint(2, size=length)

## Wczytywanie konfiguracji pana Marcina
Pliki z konfiguracjami można pobrać z https://github.com/D3M80L/CA/tree/master/phd/data.

Funkcja konwertująca ciąg bajtów na binarną reprezentację. Zwraca tablicę N pierwszych bitów:

In [4]:
def convert_configuration(configuration: bytes, N: int):
    binary_str = ''.join(f'{byte:08b}' for byte in configuration)
    return np.array([int(bit) for bit in binary_str[:N]])


Funkcja wczytująca wszystkie konfiguracje z pliku do pamięci operacyjnej:

In [5]:
def configuration_reader(file_name: str, N: int, negate: bool):
    bytes_per_configuration = math.ceil(N / 8)
    configurations = []

    with open(file_name, 'rb') as file:
        while True:
            bytes_read = file.read(bytes_per_configuration)
            if not bytes_read:
                break

            cfg = convert_configuration(bytes_read, N)
            configurations.append(cfg)
            
            if negate:
                configurations.append(1 - cfg)

    return np.stack(configurations)


- Pierwszy argument (str): nazwa pliku, zktórego chcemy pobrać konfiguracje.
- Drugi argument (int): długość konfiguracji zapisanych w pliku.
- Trzeci argument (bool): czy w wynikowej liście mają znaleźć się też "zanegowane" konfiguracje; False - konfiguracje będą miały gęstość mniejszą niż 0.5; True - konfiguracje, których gęstość jest mniejsza jaki i większa od 0.5.

Przykład użycia:

In [6]:
configurations = configuration_reader("ALL_N21.bin", 21, True)
configurations2 = configuration_reader("ALL_N21.bin", 21, False)

In [7]:
print(configurations[0:10])
print(configurations2[0:10])

[[0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 0 0 1 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 0 0 0 1 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 0 0 0 0 1 0 0 0 0 0 0]]
[[0 0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 0 1 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 0 1 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 0 1 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 0 1 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 0 1 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 0 1 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 0 1 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 0 1 1]
 [0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 0 1]]


---

# zad.7
ECAs to nie są dobre, przybliżone rozwiązania DCP. Sprawdź jak słabo
sobie radzą! To znaczy dla ustalonego N, przebadaj wszystkie ECAs dając im limit
czasu i wylicz dla jakiego % wszystkich konfiguracji początkowych
otrzymaną poprawną klasyfikację gęstości. Zaprezentuj wyniki w postaci listy par
(numer reguły, % poprawnych klasyfikacji), np.

(0, 50%)

...

(255, 50%)

Znajdź / wyróżnij regułę, która ma największy wśród ECAs % poprawnych
klasyfikacji.

In [ ]:
# gęstość konfiguracji po N krokach porównujemy z gęstością konfiguracji początkowej
# konfiguracje początkowe do badania weźmiemy z pliku ALL_N21.bin
# limit czasu to jest maksymalna długość kroków w diagramie czasoprzestrzennym (umownie 2x długość konfiguracji początkowej)

In [11]:
def dcp(rule_num, initial_config, steps):
    initial_density = sum(initial_config)/len(initial_config)
    lut = eca_get_lut(rule_num)
    config = eca_evolve_spacetime(lut, initial_config, steps)[-1]
    result_density = sum(config)/len(config)

    if initial_density < 0.5:
        if result_density < 0.5:
            return 1
        else:
            return 0
    if initial_density > 0.5:
        if result_density > 0.5:
            return 1
        else:
            return 0
    return 0


In [12]:
len(configurations)

99880

In [ ]:
configurations = configuration_reader("ALL_N21.bin", 21, True)
results = []
for rule in range(256):
    print(f'Rule {rule}...')
    l = [dcp(rule, config, 42) for config in configurations]
    results.append((rule, round(100*sum(l)/len(configurations),5)))
    print(f'({results[-1][0]}, {results[-1][1]}%)')

Rule 0...
(0, 50.0%)
Rule 1...


In [ ]:
sorted(results, key=itemgetter(1), reverse=True)[0:15]

# zad.8
Henryk Fukś pokazał, że idealne rozwiązanie DCP można uzyskać stosując ECA 184 przez "połowę czasu" a następnie ECA 232 przez następną 
połowę czasu.

Stosując wybrany “zbiór Marcina” lub zbiór wygenerowany losowo sprawdź czy odkrycie prof. Fuksia daje się odtworzyć eksperymentalnie.

Wariant (*) czy poza ECA 184 istnieją inne ECAs, które w powiązaniu z ECA 232 dają taką własność? Chodzi o inną parę ECA x, ECA 232 lub ewentualnie ECA x, ECA y, które stosowane tak jak przez prof. Fuksia rozwiązują DCP dokładnie.

## 184 - traffic right and 232 - majority

In [8]:
def dcp2(lut1, lut2, initial_config, steps):
    initial_density = sum(initial_config)/len(initial_config)
    config = eca_evolve_spacetime(lut1, initial_config, steps//2)[-1]
    result_config = eca_evolve_spacetime(lut2, config, steps//2)[-1]
    result_density = sum(result_config)/len(result_config)

    if initial_density < 0.5:
        if result_density < 0.5:
            return 1
        else:
            return 0
    if initial_density > 0.5:
        if result_density > 0.5:
            return 1
        else:
            return 0
    return 0


In [9]:
configurations = configuration_reader("ALL_N21.bin", 21, True)
lut184 = eca_get_lut(184)
lut232 = eca_get_lut(232)
l = [dcp2(lut184, lut232, config, 42) for config in configurations]
print(f'Poprawna klasyfikacja gęstości: {round(100*sum(l)/len(configurations),5)}%')

Poprawna klasyfikacja gęstości: 100.0%


## 226 - traffic left and 232 - majority

In [10]:
configurations = configuration_reader("ALL_N21.bin", 21, True)
lut226 = eca_get_lut(226)
lut232 = eca_get_lut(232)
l = [dcp2(lut226, lut232, config, 42) for config in configurations]
print(f'Poprawna klasyfikacja gęstości: {round(100*sum(l)/len(configurations),5)}%')

Poprawna klasyfikacja gęstości: 100.0%


---
Try:
- 240 right shift
- 170 left shift
- 204 identyczność
- ...

# zad.9
Zapoznaj się z wynikami z artykułu: *Mendonça, J. R. G. (2019).* **Simply modified GKL density classifiers that reach consensus faster.** *Physics Letters A, 383(19), 2264–2266.* doi:10.1016/j.physleta.2019.04.033 (https://sci-hub.se/https://www.sciencedirect.com/science/article/abs/pii/S0375960119303512?via=ihub).

Spróbuj powtórzyć eksperymenty tam wykonane dla wybranego N dla którego mamy "zbiór Marcina" badając średni czas klasyfikacji oraz % poprawnych klasyfikacji dla reguł GKL(j, k) dla kilku przypadków j, k.

Otrzymane wyniki porównaj z tymi z Zadania 7. To znaczy jak bardzo lepsze są najlepsze reguły GKL, które uda Ci się znaleźć w porównaniu do najlepszych ECAs.

# zad.10(*)
Sprawdź jak wybór zbioru testowych warunków początkowych wypływa na wyniki eksperymentów z Zadania 8.

Czy jesteś w stanie znaleźć przykład parametrów N (liczba komórek), num_conf (liczba losowanych warunków w zbiorze), p (prawdopodobieństwo 0 w 
komórce) do losowania, aby pokazać, że eliminacja duplikatów (shift-equal) ma znaczenie - tj. niechlujne dobranie "losowego" zbioru warunków początkowych zawyża / zaniża szacowaną jakość klasyfikacji.

**Podpowiedź**: Możesz porównać wyniki otrzymane na wybranym "zbiorze Marcina" z wynikami na zbiorze otrzymanym w zwykłym losowaniu dla konkretnego N.